# Dealing with FreeRiders in P2P 

- Freriders can be present in any p2p system on multiple levels. 
- We address the freeriders on the networking level, i.e. when peer don't particicpate in the protocol to save the resources. 
    - Active freeriding on the networking level - trying to decrease the quality of service from other peers and increase own QoS. 
    


## Experiment 1

**Context**: Exchanging the transactions and data, such that every peer receives it. 

**Examples**: Bitcoin, Gossip Network, DHT 


### Process and plan 

- Create an overlay with peer spreaded either random:
    * Unstructured: Gossip Network like in Bitcoin. 
    * Structured: DHT-like network with a load. Wikiedia on IPFS - patterns of the access.  
- Load on the peers: 
    * Unstructured: The load is generated from the clients. Clients produce transactions by Pareto law. Each peer must spread the transactions evenly.  
    * Structured: clients generate load by requesting files. Peer store the files unevenly but relay the traffic. Search in DHT using the Zipf law distribution 
    
    
#### Attack scenarios 

- Model one freerider: See the effect of the freeriding on one peer. How much you can save while not losing the priority;
- Model K freeriders: What is the effect on the whole system. How it will change with the number of freeriders


In [24]:
from random import choice

import matplotlib.pyplot as plt
import networkx as nx

# Define locations
from ledger0.application_handler import AccountableGossip, AccountableMessageProducer
from ledger0.crawler import CrawlerService
from ledger0.ledger import LedgerNetterWorker
from ledger0.reputation import ReputationService
from p2psimpy.config import *
from p2psimpy.consts import *
from p2psimpy.services.connection_manager import BaseConnectionManager

# Locations and latencies between the points 
class Locations(Config):
    locations = ['Ohio', 'Ireland', 'Tokyo']
    latencies = {
        'Ohio': {'Ohio': Dist('invgamma', (5.54090, 0.333305, 0.987249)),
                 'Ireland': Dist('norm', (73.6995, 1.19583092197097127)),
                 'Tokyo': Dist('norm', (156.00904977375566, 0.09469886668079797))
                },
        'Ireland':{'Ireland': Dist('invgamma', (6.4360455224301525, 0.8312748033308526, 1.086191852963273)),
                   'Tokyo': Dist('norm', (131.0275, 0.25834811785650774))
                  },
        'Tokyo': {'Tokyo':  Dist('invgamma', (11.104508341331055, 0.3371934865734555, 2.0258998705983737))}
    }
    
# Peer physical properties 
class PeerConfig(Config):
    location = Dist('sample', Locations.locations)
    bandwidth_ul = Dist( 'norm', (50*MBit, 10*MBit))
    bandwidth_dl = Dist( 'norm', (50*MBit, 10*MBit))

# Configuration used for our GossipService
class GossipConfig(Config):
    exclude_types={'client',}

# We have not two types of nodes: *peer* and *client*
def prepare_peer_types():
    return { 'peer': PeerType(PeerConfig,   {BaseConnectionManager: None,
                                             AccountableGossip: GossipConfig,
                                             LedgerNetterWorker: None,
                                             CrawlerService: None,
                                             ReputationService: None
                                             }),
             'client': PeerType(PeerConfig, (BaseConnectionManager,
                                             AccountableMessageProducer,
                                             LedgerNetterWorker,
                                             CrawlerService
                                             ))}

def prepare_topology(num_peers=25, num_clients=1):    
    # Create network topology
    G = nx.erdos_renyi_graph(num_peers, 0.4)   
    nx.relabel_nodes(G, {k: k+1 for k in G.nodes()} ,copy=False)
    
    # Connect the client node to a random peer
    client_edges = [(i, choice(list(G.nodes())))
                    for i in range(num_peers+1, num_clients+num_peers+1)]
    G.add_edges_from(client_edges)

    types_map = {k: 'peer' if k < num_peers+1 else 'client' for k in G.nodes()}
    # Assign a peer type to the peers 
    nx.set_node_attributes(G, types_map , 'type')
    return G

def visualize_peer_client_network(G):
    plt.figure(figsize=(10,10))

    # Draw client/ peer network 

    master_nodes = [n for (n,ty) in \
        nx.get_node_attributes(G,'type').items() if ty == 'peer']
    client_nodes = [n for (n,ty) in \
        nx.get_node_attributes(G,'type').items() if ty == 'client']

    pos = nx.kamada_kawai_layout(G)

    nx.draw_networkx_nodes(G, pos, nodelist=master_nodes,
                           node_color='blue', node_shape='o',
                           node_size=500)
    nx.draw_networkx_nodes(G, pos, nodelist=client_nodes,
                           node_color='green', node_shape='^',
                           node_size=100, label=1)

    nx.draw_networkx_labels(G, pos, labels={k:k for k in master_nodes},
                            font_color='w')

    nx.draw_networkx_edges(G, pos,
                           edgelist=G.subgraph(master_nodes).edges(),
                           width=1.5)
    nx.draw_networkx_edges(G, pos,
                           edgelist=G.edges(nbunch=client_nodes),
                           style='dotted')



In [25]:
from p2psimpy.simulation import BaseSimulation

#  logger_dir='new_log', enable_logger=True
sim = BaseSimulation(Locations, prepare_topology(), prepare_peer_types())
sim.run(11_000)


In [4]:
sim.peers[1].counters.receive_counters

defaultdict(float,
            {'24': 8370.0,
             '19': 7750.0,
             '5': 9610.0,
             '22': 6200.0,
             '12': 6510.0,
             '16': 4650.0,
             '6': 5580.0,
             '4': 7440.0,
             '13': 8990.0,
             '15': 8370.0})

In [21]:
sim.peers[5].storage['trust_store'].txs

{2000: {},
 4000: {'10': -6.6,
  '5': 0.0,
  '1': 6.4,
  '6': -8.2,
  '25': -3.8,
  '15': -1.8000000000000007,
  '8': 3.8,
  '21': 5.5,
  '2': 0.7,
  '9': 0.4,
  '13': 0.8},
 6000: {'10': -13.200000000000001,
  '5': 0.0,
  '1': 10.2,
  '6': -28.099999999999998,
  '25': -5.800000000000001,
  '15': -18.4,
  '8': 8.399999999999999,
  '21': 17.3,
  '2': 1.8,
  '9': 0.6,
  '13': 2.2,
  '12': -2.1999999999999993},
 8000: {'10': 5.300000000000001,
  '5': 0.0,
  '1': -8.200000000000003,
  '6': -24.5,
  '25': -21.0,
  '15': -11.999999999999998,
  '8': 21.0,
  '21': 29.1,
  '2': 1.5,
  '9': 1.6,
  '13': 1.4,
  '12': -10.7,
  '11': 3.4,
  '7': 2.3,
  '22': 2.7,
  '24': 2.5},
 10000: {'10': 13.799999999999999,
  '5': 0.0,
  '1': -24.299999999999997,
  '6': -2.0,
  '25': -27.2,
  '15': -24.599999999999998,
  '8': 29.5,
  '21': 26.900000000000002,
  '2': 3.5,
  '9': 2.7,
  '13': 2.7,
  '12': -11.600000000000001,
  '11': 3.5,
  '7': 1.9,
  '22': 2.4,
  '24': 2.5,
  '17': 3.6}}

In [23]:
sim.peers[1].storage['trust_store'].txs

{2000: {},
 4000: {'2': -15.799999999999999,
  '1': 0.0,
  '24': -3.5999999999999996,
  '5': -14.2,
  '7': -3.8,
  '6': 0.0,
  '13': -0.6000000000000001,
  '17': -8.9,
  '8': -12.9,
  '22': 2.6,
  '25': 1.1},
 6000: {'2': -19.9,
  '1': 0.0,
  '24': -6.3999999999999995,
  '5': -18.4,
  '7': -13.7,
  '6': 2.6,
  '13': -1.4000000000000004,
  '17': -13.8,
  '8': -19.599999999999998,
  '22': 5.299999999999999,
  '25': -3.1999999999999997,
  '11': 1.3000000000000007,
  '9': 1.6,
  '19': 1.6,
  '4': 1.7},
 8000: {'2': -13.399999999999999,
  '1': 0.0,
  '24': -15.299999999999999,
  '5': -23.5,
  '7': -11.8,
  '6': 4.4,
  '13': 1.0,
  '17': -6.200000000000001,
  '8': -12.900000000000002,
  '22': -5.899999999999999,
  '25': 0.6000000000000005,
  '11': 19.1,
  '9': 2.9,
  '19': 2.7,
  '4': 4.8},
 10000: {'2': -20.4,
  '1': 0.0,
  '24': 1.6000000000000014,
  '5': -16.699999999999996,
  '7': -79.0,
  '6': 5.9,
  '13': 10.5,
  '17': -17.5,
  '8': -6.800000000000001,
  '22': 4.600000000000001,
  '25'

In [30]:
import networkx as nx


for i in sim.peers:
    nx.write_edgelist(
        sim.peers[1].storage['work_graph'].G,
        'datasets/wg_p'+str(sim.peers[i])+'.gel'
    )


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/wg_pPeer_25:peer.gel'

In [12]:
sim.peers[1].storage['trust_store'].txs

{2000: {},
 4000: {'6': 8.0,
  '1': 0.0,
  '26': -23.9,
  '3': -3.4000000000000004,
  '2': -7.6,
  '11': -4.9,
  '15': -3.8000000000000007,
  '17': 3.999999999999999,
  '14': 0.29999999999999893,
  '9': -11.3,
  '23': -5.3999999999999995,
  '19': -1.299999999999999,
  '5': 1.6,
  '25': 1.8,
  '20': 9.7},
 6000: {'6': 21.3,
  '1': 0.0,
  '26': -58.8,
  '3': 2.5999999999999996,
  '2': -15.799999999999997,
  '11': 6.6,
  '15': -5.1,
  '17': 15.7,
  '14': 12.9,
  '9': -23.9,
  '23': -10.600000000000001,
  '19': -10.6,
  '5': 3.9,
  '25': 4.2,
  '20': 10.0},
 8000: {'6': 19.200000000000003,
  '1': 0.0,
  '26': -82.8,
  '3': 2.400000000000002,
  '2': -9.000000000000004,
  '11': 11.899999999999999,
  '15': 13.6,
  '17': 16.700000000000003,
  '14': -12.600000000000001,
  '9': -33.900000000000006,
  '23': 0.1999999999999993,
  '19': -3.8000000000000007,
  '5': 2.9,
  '25': 4.4,
  '20': 12.599999999999998},
 10000: {'6': 22.7,
  '1': 0.0,
  '26': -96.2,
  '3': 25.6,
  '2': 7.899999999999999,
  '

In [26]:
print(sim.peers[1].storage['work_graph'].G.number_of_edges())
print(sim.peers[2].storage['work_graph'].G.number_of_edges())
print(sim.peers[3].storage['work_graph'].G.number_of_edges())
print(sim.peers[4].storage['work_graph'].G.number_of_edges())
print(sim.peers[5].storage['work_graph'].G.number_of_edges())

47
41
40
35
35


In [27]:
sim.peers

{25: Peer_25:peer,
 24: Peer_24:peer,
 23: Peer_23:peer,
 22: Peer_22:peer,
 21: Peer_21:peer,
 20: Peer_20:peer,
 19: Peer_19:peer,
 18: Peer_18:peer,
 17: Peer_17:peer,
 16: Peer_16:peer,
 15: Peer_15:peer,
 14: Peer_14:peer,
 13: Peer_13:peer,
 12: Peer_12:peer,
 11: Peer_11:peer,
 10: Peer_10:peer,
 9: Peer_9:peer,
 8: Peer_8:peer,
 7: Peer_7:peer,
 6: Peer_6:peer,
 5: Peer_5:peer,
 4: Peer_4:peer,
 3: Peer_3:peer,
 2: Peer_2:peer,
 1: Peer_1:peer,
 26: Peer_26:client}

In [ ]:
print(sim.peers[1].storage['work_graph'].G.number_of_edges())
print(sim.peers[2].storage['work_graph'].G.number_of_edges())
print(sim.peers[3].storage['work_graph'].G.number_of_edges())
print(sim.peers[4].storage['work_graph'].G.number_of_edges())
print(sim.peers[5].storage['work_graph'].G.number_of_edges())

In [ ]:
print(sim.peers[1].storage['work_graph'].G.number_of_edges())
print(sim.peers[2].storage['work_graph'].G.number_of_edges())
print(sim.peers[3].storage['work_graph'].G.number_of_edges())
print(sim.peers[4].storage['work_graph'].G.number_of_edges())
print(sim.peers[5].storage['work_graph'].G.number_of_edges())

In [ ]:
print(sim.peers[1].storage['work_graph'].G.number_of_edges())
print(sim.peers[2].storage['work_graph'].G.number_of_edges())
print(sim.peers[3].storage['work_graph'].G.number_of_edges())
print(sim.peers[4].storage['work_graph'].G.number_of_edges())
print(sim.peers[5].storage['work_graph'].G.number_of_edges())

In [ ]:
G.number_of_edges()


### How quick we converge

*Convergance - $\delta t$ to have $|E|$ nodes*

- Random walk on ego network + Random 5 transactions on response + $500 ms$:
  $50 s$, 100 rounds of crawling
-


### Overlay enhancements
1. **An accounting system**

Count the messages bandwidth between the peers based on pairwise interaction.
The peers exchange the counters and store their views together
with signature.
2. **Sampling algorithm**

Sample the transactions from the network starting crawling the data
from ego work graph. Sampling the transactions


3. Reputation (Classification) algorithm
4. Scheduling, Routing advices and Restructuring
---
- Handler to store the messages - with a graph structure
- MessageProducer - interaction with a peer
- Sampling - a runner to collect and put in the storage.
- Reputation - a runner, periodically write a reputation scores.
- Routing advices - handler for routing. A smart router.
